In [2]:
#loading packages
import torch
import torchaudio
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F

from audio_dataloader import AudioData
from cnn import CNNNetwork
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
device=('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
#signal parmaters
sample_rate=16000
num_samples=31840
win_length =int(0.025 * sample_rate)
hop_length =int(0.01 * win_length)

#path
train_metadata = '/home/kesav/Documents/kesav/research/code_files/LibriPhrase/metadata/train_1word.csv'
test_metadata = '/home/kesav/Documents/kesav/research/code_files/LibriPhrase/metadata/test_1word.csv'
data_dir = '/home/kesav/Documents/kesav/research/code_files/LibriPhrase/database/LibriPhrase_diffspk_all'

In [5]:
#mel-filter banks coefficients
mel_spectrogram = torchaudio.transforms.MelSpectrogram(
    sample_rate=sample_rate,
    n_fft=400,
    # win_length=win_length,
    hop_length=160,
    n_mels=80
    )

#loading_data from directory
train_data=AudioData(
        train_metadata, 
        data_dir, 
        mel_spectrogram, 
        sample_rate, 
        num_samples, 
        device
    )
test_data=AudioData(
        test_metadata, 
        data_dir, 
        mel_spectrogram, 
        sample_rate, 
        num_samples, 
        device
    )

In [6]:
train_dataloader = DataLoader(train_data, batch_size=batch_size,drop_last=True, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size,drop_last=True, shuffle=True)

In [43]:
#chekcing the data shape
for i in train_dataloader:
    print(i[0].shape)
    print(i[1].shape)
    break

torch.Size([32, 1, 80, 400])
torch.Size([32])


In [2]:
#hyperparameters
batch_size = 32
log_interval = 20
num_epochs = 10
in_channels = 1
num_classes = 2
learning_rate = 0.001

In [8]:
#simple neural network
class simpleNN(nn.Module):
    def __init__(self, input, output):
        super(simpleNN, self).__init__()
        self.fc1=nn.Linear(input,50)
        self.fc2=nn.Linear(50,output)
    
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return x
    

In [72]:
# Simple CNN
class AudioEncoder(nn.Module):
    def __init__(self, in_channels=1, num_classes=2):
        super(AudioEncoder, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=in_channels,out_channels=8,kernel_size=5,stride=2,padding=1)
        
        self.bn1 = nn.BatchNorm1d(8)
        
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(
            in_channels=8,
            out_channels=16,
            kernel_size=5,
            stride=1,
            padding=1,
        )
        self.bn2 = nn.BatchNorm1d(16)
        self.fc1 = nn.Linear(16*3998, num_classes)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # print('original', x.shape)
        x = F.relu(self.bn1(self.conv1(x)))
        # print('conv1', x.shape)
        x = self.pool(x)
        # print('ppoling 1',x.shape)
        x = self.dropout(x)
        x = F.relu(self.bn2(self.conv2(x)))
        # print('conv2',x.shape)
        x = self.pool(x)
        # print('pooling 2',x.shape)
        x = x.reshape(x.shape[0], -1)
        x = self.dropout(x)
        # print('reshaping',x.shape)
        x = self.fc1(x)
        # print('fully connected',x.shape)
        return x


In [28]:


# Initialize network
model = AudioEncoder(in_channels=in_channels, num_classes=num_classes).to(device)
print(model.parameters())

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
temp={}
loss_temp={}
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            # print(x.shape)
            x = x.view(x.shape[0], x.shape[1], -1)
            # print(x.shape)
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples

In [ ]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_dataloader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        data = data.view(data.shape[0], data.shape[1], -1)
        # print(data.shape)
        # forward
        scores = model(data)
        _, predictions = scores.max(1)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent or adam step
        optimizer.step()
        
    if epoch%5==0:
        temp[epoch]=predictions
        loss_temp[epoch]=loss
    
    print(f"Accuracy on training set: {check_accuracy(train_dataloader, model)*100:.2f}")
    print(f"Accuracy on test set: {check_accuracy(test_dataloader, model)*100:.2f}")

In [1]:
print('predictions at each 10th epoch', temp)
print('\n')
print('loss at each 10th epoch', loss_temp)

NameError: name 'temp' is not defined